In [1]:
from article_dataset_builder.harvest import Harverster
from software_mentions_client.client import software_mentions_client
import requests
import tqdm

In [6]:
harvester = Harverster(config_path="config/config-harvester.json", dump_metadata=True)
#harvester.reset(True)
harvester.harvest_dois("data/comparison_full_set.csv")

  0%|          | 0/1501 [00:00<?, ?it/s]

number of articles to harvest: 1501 



  8%|▊         | 126/1501 [00:14<02:39,  8.63it/s]


ReadersFullError: mdb_txn_begin: b'MDB_READERS_FULL: Environment maxreaders limit reached' (Please use a larger Environment(max_readers=) parameter)